# Transportation model under varying vehicle capacity

## Problem Description
**Transportation Problem with Varying Vehicular Capacities**

**Objective:**

Minimize the transportation cost of goods from suppliers to consumers using different types of vehicles, each with its own capacity.

**Data:**

- Suppliers: Supplier 1, Supplier 2
- Consumers: Customer 1, Customer 2, Customer 3
- Vehicle Types: Vehicle 1, Vehicle 2

**Supply and Demand:**

- Supply from Supplier 1: 30 units
- Supply from Supplier 2: 25 units
- Demand at Customer 1: 20 units
- Demand at Customer 2: 25 units
- Demand at Customer 3: 10 units

**Transportation Costs (per unit):**

- Supplier 1 to Customer 1 using Vehicle 1: N40
- Supplier 1 to Customer 1 using Vehicle 2: N50
- Supplier 1 to Customer 2 using Vehicle 1: N60
- Supplier 1 to Customer 2 using Vehicle 2: N40
- Supplier 1 to Customer 3 using Vehicle 1: N70
- Supplier 1 to Customer 3 using Vehicle 2: N110
- Supplier 2 to Customer 1 using Vehicle 1: N40
- Supplier 2 to Customer 1 using Vehicle 2: N50
- Supplier 2 to Customer 2 using Vehicle 1: N80
- Supplier 2 to Customer 2 using Vehicle 2: N50
- Supplier 2 to Customer 3 using Vehicle 1: N60
- Supplier 2 to Customer 3 using Vehicle 2: N100

**Vehicle Capacities:**

- Capacity of Vehicle 1: 20 units
- Capacity of Vehicle 2: 25 units

**Decision Variables:**

- Let $(x_{s,c,v})$ represent the quantity of goods to be shipped from supplier $s$ to consumer $c$ using vehicle type $v$.

**Objective Function:**

Minimize the total transportation cost:

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  Minimize $\sum_{s}\sum_{c}\sum_{v} x_{s,c,v} \times transportation\_cost_{s,c,v}$

**Constraints:**

1. Supply constraints: For each supplier, the shipped quantity cannot exceed the supply:

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $\sum_{c}\sum_{v} x_{s,c,v} \le supply_{s}$ for each supplier $s$ 

2. Demand constraints: For each consumer, the received quantity must meet the demand:

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  $\sum_{c}\sum_{v} x_{s,c,v} = demand_{s}$ for each consumer $c$ 

3. Vehicle capacity constraints: For each supplier and vehicle type, the shipped quantity must not exceed the vehicle's capacity:

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  $\sum_{c}\sum_{v} x_{s,c,v} \le Vehicle\_Capacity_{v}$ for each supplier $s$  and each vehicle type $v$


In [2]:
from pulp import *

# Create a linear programming problem instance
problem = pulp.LpProblem("Transportation_Problem", LpMinimize)

# Data
suppliers = ['Supplier 1', 'Supplier 2']
consumers = ['Customer 1', 'Customer 2', 'Customer 3']
vehicle_types = ['Vehicle 1', 'Vehicle 2']

# Supply and demand
supply = {'Supplier 1': 30, 'Supplier 2': 25}
demand = {'Customer 1': 20, 'Customer 2': 25, 'Customer 3': 10}

# Transportation cost per unit
transportation_cost = {
    ('Supplier 1', 'Customer 1', 'Vehicle 1'): 40,
    ('Supplier 1', 'Customer 1', 'Vehicle 2'): 50,
    ('Supplier 1', 'Customer 2', 'Vehicle 1'): 60,
    ('Supplier 1', 'Customer 2', 'Vehicle 2'): 40,
    ('Supplier 1', 'Customer 3', 'Vehicle 1'): 70,
    ('Supplier 1', 'Customer 3', 'Vehicle 2'): 110,
    ('Supplier 2', 'Customer 1', 'Vehicle 1'): 40,
    ('Supplier 2', 'Customer 1', 'Vehicle 2'): 50,
    ('Supplier 2', 'Customer 2', 'Vehicle 1'): 80,
    ('Supplier 2', 'Customer 2', 'Vehicle 2'): 50,
    ('Supplier 2', 'Customer 3', 'Vehicle 1'): 60,
    ('Supplier 2', 'Customer 3', 'Vehicle 2'): 100,
}

# Vehicle capacities
v_capacity = {'Vehicle 1': 20, 'Vehicle 2': 25}
# Decision variables
variables = LpVariable.dicts("Shipment", 
                                  ((s, c, v) for s in suppliers for c in consumers for v in vehicle_types),
                                  lowBound=0, cat="Integer")

# Objective function
problem += lpSum(variables[s, c, v] * transportation_cost[s, c, v]
                      for s in suppliers for c in consumers for v in vehicle_types)

# Supply constraints
for s in suppliers:
    problem += lpSum(variables[s, c, v] for c in consumers for v in vehicle_types) <= supply[s]

# Demand constraints
for c in consumers:
    problem += lpSum(variables[s, c, v] for s in suppliers for v in vehicle_types) == demand[c]

# Vehicle capacity constraints
for v in vehicle_types:
    for s in suppliers:
        problem += lpSum(variables[s, c, v] for c in consumers) <= v_capacity[v]


print(problem)
# Solve the problem
problem.solve()

print(f"Problem Status: {LpStatus[problem.status]}")

# Print results
print("\nOptimal Solution:")
for s in suppliers:
    for c in consumers:
        for v in vehicle_types:
            if variables[s, c, v].value() >0:
                print(f"Ship {variables[s, c, v].value()} units of goods from {s} to {c} using {v} cost N{transportation_cost[s,c,v]} per unit")

print("\nTotal Transportation Cost:", value(problem.objective))



Transportation_Problem:
MINIMIZE
40*Shipment_('Supplier_1',_'Customer_1',_'Vehicle_1') + 50*Shipment_('Supplier_1',_'Customer_1',_'Vehicle_2') + 60*Shipment_('Supplier_1',_'Customer_2',_'Vehicle_1') + 40*Shipment_('Supplier_1',_'Customer_2',_'Vehicle_2') + 70*Shipment_('Supplier_1',_'Customer_3',_'Vehicle_1') + 110*Shipment_('Supplier_1',_'Customer_3',_'Vehicle_2') + 40*Shipment_('Supplier_2',_'Customer_1',_'Vehicle_1') + 50*Shipment_('Supplier_2',_'Customer_1',_'Vehicle_2') + 80*Shipment_('Supplier_2',_'Customer_2',_'Vehicle_1') + 50*Shipment_('Supplier_2',_'Customer_2',_'Vehicle_2') + 60*Shipment_('Supplier_2',_'Customer_3',_'Vehicle_1') + 100*Shipment_('Supplier_2',_'Customer_3',_'Vehicle_2') + 0
SUBJECT TO
_C1: Shipment_('Supplier_1',_'Customer_1',_'Vehicle_1')
 + Shipment_('Supplier_1',_'Customer_1',_'Vehicle_2')
 + Shipment_('Supplier_1',_'Customer_2',_'Vehicle_1')
 + Shipment_('Supplier_1',_'Customer_2',_'Vehicle_2')
 + Shipment_('Supplier_1',_'Customer_3',_'Vehicle_1')
 + Shipm

## Expalanation of Solution
**Total Transportation Cost:**
The total transportation cost is the sum of the costs associated with shipping goods from each supplier to each consumer using different vehicle capacity. The objective function is designed to minimize this total cost.

As Obtained, the calculated total transportation cost is N2450, it means that the chosen shipping plan minimizes the cost of transporting the goods from suppliers to consumers, taking into account the transportation cost per unit for each route.

**Optimality:**
The solution is considered optimal when the transportation cost cannot be further reduced while satisfying all constraints. The optimization algorithm ensures that the chosen shipping plan is the most cost-efficient considering the provided data and constraints.